In [1]:
import numpy as np
import pandas as pd
import psycopg2
import os

In [2]:
cwd = os.getcwd()
path = os.path.join(cwd,r'Genesis_normal.csv')

df = pd.read_csv(path)
df = df.loc[:,['MotorData.ActCurrent','MotorData.ActSpeed','MotorData.IsAcceleration','MotorData.IsForce']] #select important variables
df.columns = df.columns.str.replace('.','_')
df

C:\Users\MP\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  import sys


,MotorData_ActCurrent,MotorData_ActSpeed,MotorData_IsAcceleration,MotorData_IsForce
0,384,0,0,84
1,380,0,0,83
2,380,0,0,83
3,393,0,0,84
4,384,0,0,83
...,...,...,...,...
7035,350,0,0,75
7036,344,0,0,78
7037,358,0,0,77
7038,358,0,0,77


In [1]:
import psycopg2
from io import StringIO

# Connection parameters
param_dic = {
    "host"      : "localhost",
    "database"  : "sensordata",
    "user"      : "postgres",
    "password"  : "JNt8hSW4l+MJ]p;.g[WG"
}

#Credit to Naysan Saran. https://naysan.ca/2020/06/21/pandas-to-postgresql-using-psycopg2-copy_from/

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn


def copy_from_stringio(conn, df, table):
    """
    Here we are going save the dataframe in memory 
    and use copy_from() to copy it to the table
    """
    # save dataframe to an in memory buffer
    buffer = StringIO()
    df.to_csv(buffer, index_label='id', header=False)
    buffer.seek(0)

    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:

        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_stringio() done")
    cursor.close()

In [2]:
conn = connect(param_dic) # connect to the database
cursor = conn.cursor() # set cursor

# CREATE TABLE
list1 = list(df.keys()) #use column labels for SQL query
string1 = "CREATE TABLE genesis_demonstrator (id int"
for label in list1:
    string1 += (", " + label + " decimal")
string1 = string1 + ");"
print(string1)
cursor.execute(string1) #execute SQL query

copy_from_stringio(conn, df, "genesis_demonstrator") # copy the dataframe to SQL
conn.close() # close the connection

Connecting to the PostgreSQL database...
Connection successful


NameError: name 'df' is not defined